In [ ]:
# Upgrade Oracle ADS to pick up the latest preview version to maintain compatibility with Oracle Cloud Infrastructure.

!odsc conda install --uri https://objectstorage.us-ashburn-1.oraclecloud.com/p/qnzzHQPGQYghdyH206yDk25MZH1FaMGdNNhKUl74BhRsW4muvFyGViKIqpxgnxI3/n/ociodscdev/b/ads_conda_pack_builds/o/PySpark_3/teamcity_20230512_084146_38972446/f227145b7ee5fc1c73a69ebaa671b81e/PySpark_3.2_and_Feature_Store.tar.gz

Oracle Data Science service sample notebook.

Copyright (c) 2022 Oracle, Inc. All rights reserved. Licensed under the [Universal Permissive License v 1.0](https://oss.oracle.com/licenses/upl).

***

# <font color="red">Feature store handling querying operations</font>
<p style="margin-left:10%; margin-right:10%;">by the <font color="teal">Oracle Cloud Infrastructure Data Science Service.</font></p>

---
# Overview:
---
Managing many datasets, data-sources and transformations for machine learning is complex and costly. Poorly cleaned data, data issues, bugs in transformations, data drift and training serving skew all leads to increased model development time and worse model performance. Here, feature store is well positioned to solve many of the problems since it provides a centralised way to transform and access data for training and serving time and helps defines a standardised pipeline for ingestion of data and querying of data. This notebook demonstrates how to use feature store within a long lasting [Oracle Cloud Infrastructure Data Flow](https://docs.oracle.com/en-us/iaas/data-flow/using/home.htm) cluster.

Compatible conda pack: [PySpark 3.2 and Feature store](https://docs.oracle.com/iaas/data-science/using/conda-pyspark-fam.htm) for CPU on Python 3.8

## Contents:

- <a href="#concepts">1. Introduction</a>
- <a href='#pre-requisites'>1. Pre-requisites</a>
    - <a href='#policies'>2.1 Policies</a>
    - <a href='#prerequisites_authentication'>2.2 Authentication</a>
    - <a href='#prerequisites_variables'>2.3 Variables</a>
- <a href='#featurestore_querying'>3. Feature store querying</a>
    - <a href='#load_featuregroup'>3.1. Load feature groups</a>
    - <a href='#explore_featuregroup'>3.2. Explore feature groups</a>
    - <a href='#select_subset_featuregroup'>3.3. Select subset of features</a>
    - <a href='#explore_featuregroup'>3.4. Explore feature groups</a>
    - <a href='#filter_featuregroup'>3.5. Filter feature groups</a>
    - <a href='#create_dataset'>3.6. Create dataset from multiple or one feature group</a>
    - <a href='#query_dataset'>3.7 Query dataset</a>
- <a href='#ref'>4. References</a>

---

**Important:**

Placeholder text for required values are surrounded by angle brackets that must be removed when adding the indicated content. For example, when adding a database name to `database_name = "<database_name>"` would become `database_name = "production"`.

---

<a id="concepts"></a>
# 1. Introduction

Oracle feature store is a stack based solution that is deployed in the customer enclave using OCI resource manager. Customer can stand up the service with infrastructure in their own tenancy. The service consists of API which are deployed in customer tenancy using resource manager.

The following are some key terms that will help you understand OCI Data Science Feature Store:


* **Feature Vector**: Set of feature values for any one primary/identifier key. Eg.  All/subset of  features of customer id '2536' can be called as one feature vector.
* **Feature**: A feature is an individual measurable property or characteristic of a phenomenon being observed.
* **Entity**: An entity is a group of semantically related features.
* **Datasource**: Features are engineered from raw data stored in various data sources (e.g. object storage, Oracle Database, Oracle MySQL, etc).
* **Feature Group** - A feature group is an object that represents a logical group of time-series feature data as it is found in a datasource.
* **Dataset**: Datasets are created from features stored in the feature store service and are used to train models and to perform online model inference.

<a id='pre-requisites'></a>
# 2. Pre-requisites

Data Flow Sessions are accessible through the following conda environment:

* **PySpark 3.2, Feature store 1.0 and Data Flow 1.0 (pyspark32_p38_cpu_feature_store_v1)**

The [Data Catalog Hive Metastore](https://docs.oracle.com/en-us/iaas/data-catalog/using/metastore.htm) provides schema definitions for objects in structured and unstructured data assets. The Metastore is the central metadata repository to understand tables backed by files on object storage. You can customize `pyspark32_p38_cpu_feature_store_v1`, publish it, and use it as a runtime environment for a Data Flow session cluster. The metastore id of hive metastore is tied to feature store construct of feature store service.

<a id='policies'></a>
### 2.1. Policies
This section covers the creation of dynamic groups and policies needed to use the service.

* [Data Flow Policies](https://docs.oracle.com/iaas/data-flow/using/policies.htm/)
* [Data Catalog Metastore Required Policies](https://docs.oracle.com/en-us/iaas/data-catalog/using/metastore.htm)
* [Getting Started with Data Flow](https://docs.oracle.com/iaas/data-flow/using/dfs_getting_started.htm)
* [About Data Science Policies](https://docs.oracle.com/iaas/data-science/using/policies.htm)

<a id="prerequisites_authentication"></a>
### 2.2. Authentication
The [Oracle Accelerated Data Science SDK (ADS)](https://docs.oracle.com/iaas/tools/ads-sdk/latest/index.html) controls the authentication mechanism with the notebook cluster.<br>
To setup authentication use the ```ads.set_auth("resource_principal")``` or ```ads.set_auth("api_key")```.

In [1]:
import ads
ads.set_auth(auth="api_key", client_kwargs={"service_endpoint": "http://localhost:21000/20230101"})

<a id="prerequisites_variables"></a>
### 2.3. Variables
To run this notebook, you must provide some information about your tenancy configuration. To create and run a feature store, you must specify a `<compartment_id>` and bucket `<metastore_id>` for offline feature store.

In [2]:
import os

compartment_id = "ocid1.tenancy.oc1..aaaaaaaa462hfhplpx652b32ix62xrdijppq2c7okwcqjlgrbknhgtj2kofa"
metastore_id = "ocid1.datacatalogmetastore.oc1.iad.amaaaaaabiudgxyap7tizm4gscwz7amu7dixz7ml3mtesqzzwwg3urvvdgua"

<a id="featurestore_querying"></a>
# 3. Feature group querying
By default the **PySpark 3.2, Feature store and Data Flow** conda environment includes pre-installed [great-expectations](https://legacy.docs.greatexpectations.io/en/latest/reference/core_concepts/validation.html) and [deeque](https://github.com/awslabs/deequ) libraries. The joining functionality is heavily inspired by the APIs used by Pandas to merge, join or filter DataFrames. The APIs allow you to specify which features to select from which feature group, how to join them and which features to use in join conditions.

In [3]:
import pandas as pd
from ads.feature_store.feature_store import FeatureStore
from ads.feature_store.feature_group import FeatureGroup
from ads.feature_store.dataset import Dataset
from ads.feature_store.common.enums import DatasetIngestionMode

  if LooseVersion(pandas.__version__) < LooseVersion(minimum_pandas_version):

  if LooseVersion(pyarrow.__version__) < LooseVersion(minimum_pyarrow_version):

  LooseVersion(pyarrow.__version__) >= LooseVersion("2.0.0")

  warnings.warn(

  if LooseVersion(pd.__version__) >= LooseVersion("1.3.0"):

ERROR:logger:Please set env variable SPARK_VERSION
INFO:logger:Using deequ: com.amazon.deequ:deequ:1.2.2-spark-3.0


<a id="data_exploration"></a>
### 3.1. Exploration of data in feature store

In [4]:
flights_df = pd.read_csv("~/Downloads/flights.csv")
flights_df.head()

  flights_df = pd.read_csv("~/Downloads/flights.csv")



,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,...,408.0,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,...,741.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,...,811.0,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,...,756.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,...,259.0,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
airports_df = pd.read_csv("~/Downloads/airports.csv")
airports_df.head()

,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
0,ABE,Lehigh Valley International Airport,Allentown,PA,USA,40.65236,-75.44040
1,ABI,Abilene Regional Airport,Abilene,TX,USA,32.41132,-99.68190
2,ABQ,Albuquerque International Sunport,Albuquerque,NM,USA,35.04022,-106.60919
3,ABR,Aberdeen Regional Airport,Aberdeen,SD,USA,45.44906,-98.42183
4,ABY,Southwest Georgia Regional Airport,Albany,GA,USA,31.53552,-84.19447


In [6]:
airlines_df = pd.read_csv("~/Downloads/airlines.csv")
airlines_df.head()

,IATA_CODE,AIRLINE
0,UA,United Air Lines Inc.
1,AA,American Airlines Inc.
2,US,US Airways Inc.
3,F9,Frontier Airlines Inc.
4,B6,JetBlue Airways


<a id="load_featuregroup"></a>
### 3.2. Create feature store logical entities

#### 3.2.1 Feature Store
Feature store is the top level entity for feature store service

In [7]:
feature_store_resource = (
    FeatureStore().
    with_description("Data consisting of flights").
    with_compartment_id(compartment_id).
    with_display_name("flights details").
    with_offline_config(metastore_id=metastore_id)
)

<a id="create_feature_store"></a>
##### Create Feature Store

Call the ```.create()``` method of the Feature store instance to create a feature store.

In [8]:
feature_store = feature_store_resource.create()
feature_store

kind: featurestore
spec:
  compartmentId: ocid1.tenancy.oc1..aaaaaaaa462hfhplpx652b32ix62xrdijppq2c7okwcqjlgrbknhgtj2kofa
  description: Data consisting of flights
  displayName: flights details
  id: AF2A3D458EED647F3125C51FA0BFB30C
  offlineConfig:
    metastoreId: ocid1.datacatalogmetastore.oc1.iad.amaaaaaabiudgxyap7tizm4gscwz7amu7dixz7ml3mtesqzzwwg3urvvdgua
type: featureStore

#### 3.2.2 Entity
An entity is a group of semantically related features.

In [9]:
entity = feature_store.create_entity(
    display_name="Flight details",
    description="description for flight details"
)
entity

kind: entity
spec:
  compartmentId: ocid1.tenancy.oc1..aaaaaaaa462hfhplpx652b32ix62xrdijppq2c7okwcqjlgrbknhgtj2kofa
  description: description for flight details
  featureStoreId: AF2A3D458EED647F3125C51FA0BFB30C
  id: A9CF6D8FA48651D156744107746AF78F
  name: Flight details
type: entity

#### 3.2.3 Feature group
A feature group is an object that represents a logical group of time-series feature data as it is found in a datasource.

<a id="create_feature_group_flights"></a>
##### Flights Feature Group

Create feature group for flights

In [10]:
feature_group_flights = (
    FeatureGroup()
    .with_feature_store_id(feature_store.id)
    .with_primary_keys(["FLIGHT_NUMBER"])
    .with_name("flights_feature_group")
    .with_entity_id(entity.id)
    .with_compartment_id(compartment_id)
    .with_schema_details_from_dataframe(flights_df)
)

In [11]:
feature_group_flights.create()

kind: FeatureGroup
spec:
  compartmentId: ocid1.tenancy.oc1..aaaaaaaa462hfhplpx652b32ix62xrdijppq2c7okwcqjlgrbknhgtj2kofa
  entityId: A9CF6D8FA48651D156744107746AF78F
  featureStoreId: AF2A3D458EED647F3125C51FA0BFB30C
  id: D4CBEA0B8E8E23CB560750DB3022D89E
  inputFeatureDetails:
  - featureType: INTEGER
    name: YEAR
    orderNumber: 1
  - featureType: INTEGER
    name: MONTH
    orderNumber: 2
  - featureType: INTEGER
    name: DAY
    orderNumber: 3
  - featureType: INTEGER
    name: DAY_OF_WEEK
    orderNumber: 4
  - featureType: STRING
    name: AIRLINE
    orderNumber: 5
  - featureType: INTEGER
    name: FLIGHT_NUMBER
    orderNumber: 6
  - featureType: STRING
    name: TAIL_NUMBER
    orderNumber: 7
  - featureType: STRING
    name: ORIGIN_AIRPORT
    orderNumber: 8
  - featureType: STRING
    name: DESTINATION_AIRPORT
    orderNumber: 9
  - featureType: INTEGER
    name: SCHEDULED_DEPARTURE
    orderNumber: 10
  - featureType: FLOAT
    name: DEPARTURE_TIME
    orderNumber: 11

In [12]:
os.environ["DEVELOPER_MODE"] = "True"

In [13]:
feature_group_flights.materialise(flights_df)

:: loading settings :: url = jar:file:/Users/kshitizlohia/IdeaProjects/oracle/feature-store/advanced-ds/venv/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/kshitizlohia/.ivy2/cache
The jars for the packages stored in: /Users/kshitizlohia/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-e865a4e0-a3ac-4af8-814e-278888fbf345;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.3.0 in central
	found io.delta#delta-storage;2.3.0 in central
	found org.antlr#antlr4-runtime;4.8 in local-m2-cache
:: resolution report :: resolve 166ms :: artifacts dl 9ms
	:: modules in use:
	io.delta#delta-core_2.12;2.3.0 from central in [default]
	io.delta#delta-storage;2.3.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from local-m2-cache in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      defau

23/05/15 15:25:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


  if LooseVersion(pandas.__version__) < LooseVersion(minimum_pandas_version):

  for column, series in pdf.iteritems():

  for column, series in pdf.iteritems():



ERROR - Exception
Traceback (most recent call last):
  File "/Users/kshitizlohia/IdeaProjects/oracle/feature-store/advanced-ds/ads/feature_store/execution_strategy/spark/spark_execution.py", line 76, in ingest_feature_definition
    self._save_offline_dataframe(dataframe, feature_group, feature_group_job)
  File "/Users/kshitizlohia/IdeaProjects/oracle/feature-store/advanced-ds/ads/feature_store/execution_strategy/spark/spark_execution.py", line 183, in _save_offline_dataframe
    raw_schema = self.spark_engine.convert_from_pandas_to_spark_dataframe(
  File "/Users/kshitizlohia/IdeaProjects/oracle/feature-store/advanced-ds/ads/feature_store/execution_strategy/engine/spark_engine.py", line 154, in convert_from_pandas_to_spark_dataframe
    return self.spark.createDataFrame(dataframe)
  File "/Users/kshitizlohia/IdeaProjects/oracle/feature-store/advanced-ds/venv/lib/python3.10/site-packages/pyspark/sql/session.py", line 891, in createDataFrame
    return super(SparkSession, self).createD

<a id="create_feature_group_airport"></a>
##### Airport Feature Group

Create feature group for airport

In [22]:
feature_group_airports = (
    FeatureGroup()
    .with_feature_store_id(feature_store.id)
    .with_primary_keys(["IATA_CODE"])
    .with_name("airport_feature_group")
    .with_entity_id(entity.id)
    .with_compartment_id(compartment_id)
    .with_schema_details_from_dataframe(airports_df)
)

In [23]:
feature_group_airports.create()

kind: FeatureGroup
spec:
  compartmentId: ocid1.tenancy.oc1..aaaaaaaa462hfhplpx652b32ix62xrdijppq2c7okwcqjlgrbknhgtj2kofa
  entityId: A9CF6D8FA48651D156744107746AF78F
  featureStoreId: AF2A3D458EED647F3125C51FA0BFB30C
  id: FD1325A83A2068800CDE45E68D278025
  inputFeatureDetails:
  - featureType: STRING
    name: IATA_CODE
    orderNumber: 1
  - featureType: STRING
    name: AIRPORT
    orderNumber: 2
  - featureType: STRING
    name: CITY
    orderNumber: 3
  - featureType: STRING
    name: STATE
    orderNumber: 4
  - featureType: STRING
    name: COUNTRY
    orderNumber: 5
  - featureType: FLOAT
    name: LATITUDE
    orderNumber: 6
  - featureType: FLOAT
    name: LONGITUDE
    orderNumber: 7
  name: airport_feature_group1
  primaryKeys:
    items:
    - name: IATA_CODE
  statisticsConfig:
    isEnabled: true
type: featureGroup

In [25]:
feature_group_airports.materialise(airports_df)

23/05/15 15:31:44 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers
23/05/15 15:31:44 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 84.47% for 8 writers
23/05/15 15:31:46 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers


23/05/15 15:31:50 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


23/05/15 15:31:53 WARN HiveExternalCatalog: Couldn't find corresponding Hive SerDe for data source provider delta. Persisting data source table `a9cf6d8fa48651d156744107746af78f`.`airport_feature_group1` into Hive metastore in Spark SQL specific format, which is NOT compatible with Hive.


<a id="create_feature_group_airlines"></a>
##### Airlines Feature Group

Create feature group for airlines

In [26]:
feature_group_airlines = (
    FeatureGroup()
    .with_feature_store_id(feature_store.id)
    .with_primary_keys(["IATA_CODE"])
    .with_name("airlines_feature_group")
    .with_entity_id(entity.id)
    .with_compartment_id(compartment_id)
    .with_schema_details_from_dataframe(airlines_df)
)

In [27]:
feature_group_airlines.create()

kind: FeatureGroup
spec:
  compartmentId: ocid1.tenancy.oc1..aaaaaaaa462hfhplpx652b32ix62xrdijppq2c7okwcqjlgrbknhgtj2kofa
  entityId: A9CF6D8FA48651D156744107746AF78F
  featureStoreId: AF2A3D458EED647F3125C51FA0BFB30C
  id: 89A0CDE87CA616E4BE814A5F07631459
  inputFeatureDetails:
  - featureType: STRING
    name: IATA_CODE
    orderNumber: 1
  - featureType: STRING
    name: AIRLINE
    orderNumber: 2
  name: airlines_feature_group
  primaryKeys:
    items:
    - name: IATA_CODE
  statisticsConfig:
    isEnabled: true
type: featureGroup

In [28]:
feature_group_airlines.materialise(airlines_df)

23/05/15 15:32:40 WARN HiveExternalCatalog: Couldn't find corresponding Hive SerDe for data source provider delta. Persisting data source table `a9cf6d8fa48651d156744107746af78f`.`airlines_feature_group` into Hive metastore in Spark SQL specific format, which is NOT compatible with Hive.


<a id="explore_featuregroup"></a>
### 3.3. Explore feature groups

In [30]:
feature_group_airlines.select()

ERROR - Exception
Traceback (most recent call last):
  File "/Users/kshitizlohia/IdeaProjects/oracle/feature-store/advanced-ds/venv/lib/python3.10/site-packages/IPython/core/formatters.py", line 223, in catch_format_error
    r = method(self, *args, **kwargs)
  File "/Users/kshitizlohia/IdeaProjects/oracle/feature-store/advanced-ds/venv/lib/python3.10/site-packages/IPython/core/formatters.py", line 708, in __call__
    printer.pretty(obj)
  File "/Users/kshitizlohia/IdeaProjects/oracle/feature-store/advanced-ds/venv/lib/python3.10/site-packages/IPython/lib/pretty.py", line 410, in pretty
    return _repr_pprint(obj, self, cycle)
  File "/Users/kshitizlohia/IdeaProjects/oracle/feature-store/advanced-ds/venv/lib/python3.10/site-packages/IPython/lib/pretty.py", line 778, in _repr_pprint
    output = repr(obj)
  File "/Users/kshitizlohia/IdeaProjects/oracle/feature-store/advanced-ds/ads/jobs/builders/base.py", line 135, in __repr__
    return self.to_yaml()
  File "/Users/kshitizlohia/Idea

In [31]:
feature_group_airlines.select().show()

+---------+--------------------+
|IATA_CODE|             AIRLINE|
+---------+--------------------+
|       EV|Atlantic Southeas...|
|       HA|Hawaiian Airlines...|
|       UA|United Air Lines ...|
|       WN|Southwest Airline...|
|       DL|Delta Air Lines Inc.|
|       MQ|American Eagle Ai...|
|       VX|      Virgin America|
|       OO|Skywest Airlines ...|
|       AS|Alaska Airlines Inc.|
|       F9|Frontier Airlines...|
+---------+--------------------+
only showing top 10 rows



<a id="select_subset_featuregroup"></a>
### 3.4. Select subset of features

In [ ]:
feature_group_flights.select(['YEAR', 'MONTH', 'DATE', 'FLIGHT_NUMBER'])

In [32]:
feature_group_airlines.select(['IATA_CODE', 'AIRLINE']).show()

+---------+--------------------+
|IATA_CODE|             AIRLINE|
+---------+--------------------+
|       EV|Atlantic Southeas...|
|       HA|Hawaiian Airlines...|
|       UA|United Air Lines ...|
|       WN|Southwest Airline...|
|       DL|Delta Air Lines Inc.|
|       MQ|American Eagle Ai...|
|       VX|      Virgin America|
|       OO|Skywest Airlines ...|
|       AS|Alaska Airlines Inc.|
|       F9|Frontier Airlines...|
+---------+--------------------+
only showing top 10 rows



<a id="filter_featuregroup"></a>
### 3.5. Filter feature groups

In [ ]:
feature_group_airlines.filter(feature_group_airlines.CANCELLED == 0).show()

<a id="join_featuregroup"></a>
### 3.6. Apply joins on feature group
As in Pandas, if the feature has the same name on both feature groups, then you can use the `on=[]` paramter. If they have different names, then you can use the `left_on=[]` and `right_on=[]` paramters:

In [ ]:
query = feature_group_flights.select()\
                .join(feature_group_airlines.select(), left_on=['ORIGIN_AIRPORT'], right_on=['IATA_CODE'])\
                .join(feature_group_airports.select(), left_on=['AIRLINE'], right_on=['IATA_CODE'])
query.show(5)

<a id="create_dataset"></a>
### 3.7 Create dataset
A dataset is a collection of feature snapshots that are joined together to either train a model or perform model inference.

In [ ]:
dataset = Dataset()\
    .with_description("Combined dataset for flights")\
    .with_compartment_id(compartment_id)\
    .with_display_name("Flight dataset")\
    .with_entity_id(entity.id)\
    .with_dataset_ingestion_mode(DatasetIngestionMode.SQL)\
    .with_feature_store_id(feature_store.id)\
    .with_query(query)

In [ ]:
dataset.materialise()

In [ ]:
dataset_df = dataset.to_dataframe()

<a id="query_dataset"></a>
### 3.8 Query dataset
Feature store provides a DataFrame API to ingest data into the Feature Store. You can also retrieve feature data in a DataFrame, that can either be used directly to train models or materialized to file(s) for later use to train models

In [ ]:
query = dataset_df.select(["IATA_CODE", "ORIGIN_AIRPORT"])
query.show()

<a id="sql_query"></a>
### 3.9 SQL query
Feature store provides a way to query feature store using free flow query. User need to mention `entity id` as the database name and `feature group name` as the table name to query feature store. This functionality can be useful if you need to express more complex queries for your use case

In [ ]:
entity_id = entity.id

sql = (f"SELECT flights_feature_group.* "
       f"FROM {entity_id}.flights_feature_group "
       f"JOIN {entity_id}.airport_feature_group "
       f"ON {entity_id}.flights_feature_group.ORIGIN_AIRPORT={entity_id}.airport_feature_group.IATA_CODE "
       f"JOIN {entity_id}.flights_feature_group.AIRLINE={entity_id}.airlines_feature_group.IATA_CODE ")

In [ ]:
feature_store.sql(sql)

<a id='ref'></a>
# References

- [ADS Library Documentation](https://accelerated-data-science.readthedocs.io/en/latest/index.html)
- [Data Science YouTube Videos](https://www.youtube.com/playlist?list=PLKCk3OyNwIzv6CWMhvqSB_8MLJIZdO80L)
- [OCI Data Science Documentation](https://docs.cloud.oracle.com/en-us/iaas/data-science/using/data-science.htm)
- [Oracle Data & AI Blog](https://blogs.oracle.com/datascience/)